In [41]:
import pathlib
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, Float, String, Date, text
from sqlalchemy.orm import Session
from sqlalchemy import inspect, func

# Create path object using pathlib
This will ensure this file will run for everyone regardless of os
This group contains folks running on mac, windows, and linux.    

In [42]:
cwd = pathlib.Path.cwd()

if cwd.name == 'Project_3_Group_7':
    print("Path is project root")
else:
    print("Please correct current working directory to the project root")

Path is project root


In [43]:
resources_path = pathlib.PurePath(cwd, 'Resources')
resources_path

PurePosixPath('/home/mox/Documents/coding_projects/bootcamp_local/Homeworks/Project_3_Group_7/Resources')

In [44]:
db_path = pathlib.PurePath(cwd, 'db')
db_path

PurePosixPath('/home/mox/Documents/coding_projects/bootcamp_local/Homeworks/Project_3_Group_7/db')

# Create the db session
Can not use automap_base because primary key is a composite key 

https://python-code.dev/articles/132521261

In [45]:
engine = sqlalchemy.create_engine(f'sqlite:///{db_path}/data.sqlite')
Base = declarative_base()

/tmp/ipykernel_31340/1502287200.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [46]:
inspector = inspect(engine)
tables = inspector.get_table_names()
tables

['steam_twitch_agg', 'tags', 'twitch_monthly']

In [47]:
steam_twitch_tags_table = tables[0]
twitch_monthly_table = tables[2]
tags_table = tables[1]

In [48]:
class steam_twitch_agg(Base):
    __tablename__ = steam_twitch_tags_table
    # https://stackoverflow.com/questions/19129289/how-to-define-composite-primary-key-in-sqlalchemy
    # indicates that this syntax should make a composite primary key
    app_id = Column(Integer, primary_key=True)
    tag = Column(String(128), primary_key=True)
    title = Column(String(128))
    # These two will need to be changed to Float 
    hours_watched = Column(Float)
    hours_streamed = Column(Float)
    average_streamers = Column(Integer)
    average_viewers = Column(Integer)
    average_channels = Column(Integer)
    release_date = Column(Date)
    reviews_total = Column(Integer)
    review_avg_percent = Column(Integer)
    launch_price_cents = Column(Integer)
    dataset_est_rev_cents = Column(Integer)

class twitch_monthly(Base):
    __tablename__ = twitch_monthly_table
    app_id = Column(Integer, primary_key=True)
    title = Column(String(128))
    rank = Column(Integer)
    month = Column(Integer, primary_key=True)
    year = Column(Integer, primary_key=True)
    hours_watched = Column(Integer)
    hours_streamed = Column(Integer)
    peak_viewers = Column(Integer)
    peak_channels = Column(Integer)
    streamers = Column(Integer)
    average_viewers = Column(Integer)
    average_channels = Column(Integer)

class steam_tags(Base):
    __tablename__ = tags_table
    app_id = Column(Integer, primary_key=True)
    tag = Column(String(128), primary_key=True)

In [49]:
Base.metadata.create_all(engine)

In [50]:
session = Session(bind= engine)

# Queries

## Query for genre tags for both games

LawBreakers is not popular enough to be in the database.

steam genre tags copied manually from steamdb: https://steamdb.info/app/350280/

In [51]:
lawbreakers_tags = ["Action", "FPS", "Multiplayer", "Shooter", "Arena Shooter", "First-Person", "Fast-Paced", "Sci-fi", "Futuristic", "Online Co-op", "Competative", "Team-Based", "Co-op", "Difficult", "Class-Based", "Great Soundtrack", "Memes", "MOBA", "Strategy", "Indie"]

In [52]:
statement = text("SELECT app_id, title, avg_hours_streamed FROM steam_twitch_agg WHERE LOWER(title) LIKE 'hunt%' ORDER BY avg_hours_watched DESC;")
result = session.execute(statement).all()
hunt_S_app_id = result[0][0]
hunt_S_app_id

594650

In [53]:
# app_id 594650 is Hunt:Showdown 1896
# Detemined by running the following query in sqlite browser 
# SELECT app_id, title
#   FROM steam_twitch_agg
#  WHERE LOWER(title) like 'hunt%'

statement = text("SELECT tag FROM tags WHERE app_id IS 594650;")
result = session.execute(statement).all()
hunt_tags = [val[0] for val in result]
hunt_tags

['Open World',
 'Tactical',
 'Atmospheric',
 'Multiplayer',
 'FPS',
 'Perma Death',
 'Shooter',
 'PvP',
 'Survival Horror',
 'Horror',
 'Difficult',
 'Online Co Op',
 'Co op',
 'Team Based',
 'Action',
 'Gore',
 'Zombies',
 'First Person']

# Shared Genre tags betweem Hunt and Lawbreakers used as genre filter for the dataset

In [54]:
shared_tags = [val for val in hunt_tags if val in lawbreakers_tags]
shared_tags

['Multiplayer', 'FPS', 'Shooter', 'Difficult', 'Action']

### Examine 3+ shared genre tags

In [55]:
statement = text(f"""SELECT st.*
FROM steam_twitch_agg AS st
JOIN tags as t
ON st.app_id = t.app_id
WHERE t.tag IN ('Multiplayer', 'FPS', 'Shooter', 'Difficult', 'Action')
GROUP BY st.app_id 
HAVING COUNT(st.app_id) > 2 
ORDER BY st.avg_hours_streamed DESC;""")
result = session.execute(statement).all()
games_sharing_4_tags = result
print(len(games_sharing_4_tags))
games_sharing_4_tags

215


[(302, 271590, 'Grand Theft Auto V', 1123643, 1322782, 89.85, 64115728, 126038, 87926, 1539, '2015-04-13', 2999, 3967023218),
 (127, 10, 'Counter-Strike', 913781, 137421, 97.0, 53172684, 141632, 72571, 1248, '2000-11-01', 999, 137283579),
 (128, 730, 'Counter-Strike: Global Offensive', 906847, 7382695, 88.0, 48489362, 123006, 66604, 1243, '2012-08-21', 1499, 11066659805),
 (163, 381210, 'Dead by Daylight', 738369, 486959, 80.0, 13545274, 68595, 18559, 1011, '2016-06-14', 1999, 973431041),
 (174, 1085660, 'Destiny 2', 565819, 555551, 82.0, 7174193, 53869, 9832, 775, '2019-10-01', 3499, 1943872949),
 (198, 570, 'Dota 2', 509145, 2017009, 82.0, 42350161, 36145, 58036, 697, '2013-07-09', 2999, 6049009991),
 (517, 578080, 'PUBG: BATTLEGROUNDS', 495358, 2201296, 57.0, 9725767, 47808, 13337, 679, '2017-12-21', 2999, 6601686704),
 (213, 1245620, 'ELDEN RING', 334744, 520024, 92.0, 6275674, 48378, 8704, 463, '2022-02-24', 5999, 3119623976),
 (581, 1196590, 'Resident Evil Village', 294413, 64873

### Filter for 4+ shared genre tags

In [56]:
# Statement adapted from sqlalchemy exmaple https://stackoverflow.com/questions/13349832/sqlalchemy-filter-to-match-all-instead-of-any-values-in-list
statement = text(f"""SELECT st.*
FROM steam_twitch_agg AS st
JOIN tags as t
ON st.app_id = t.app_id
WHERE t.tag IN ('Multiplayer', 'FPS', 'Shooter', 'Difficult', 'Action')
GROUP BY st.app_id 
HAVING COUNT(st.app_id) > 3 
ORDER BY st.avg_hours_streamed DESC;""")
result = session.execute(statement).all()
games_sharing_4_tags = result
print(len(games_sharing_4_tags))
games_sharing_4_tags

85


[(127, 10, 'Counter-Strike', 913781, 137421, 97.0, 53172684, 141632, 72571, 1248, '2000-11-01', 999, 137283579),
 (128, 730, 'Counter-Strike: Global Offensive', 906847, 7382695, 88.0, 48489362, 123006, 66604, 1243, '2012-08-21', 1499, 11066659805),
 (174, 1085660, 'Destiny 2', 565819, 555551, 82.0, 7174193, 53869, 9832, 775, '2019-10-01', 3499, 1943872949),
 (517, 578080, 'PUBG: BATTLEGROUNDS', 495358, 2201296, 57.0, 9725767, 47808, 13337, 679, '2017-12-21', 2999, 6601686704),
 (569, 1174180, 'Red Dead Redemption 2', 236684, 394937, 90.0, 3104279, 30149, 4253, 324, '2019-12-05', 5999, 2369227063),
 (57, 924970, 'Back 4 Blood', 232005, 36130, 66.0, 6077030, 38193, 8202, 313, '2021-10-12', 5999, 216743870),
 (754, 1286680, "Tiny Tina's Wonderlands", 201391, 9094, 76.0, 3259927, 30371, 4441, 275, '2022-06-23', 7999, 72742906),
 (597, 252490, 'Rust', 183374, 775223, 87.0, 10434473, 18355, 14292, 251, '2018-02-08', 3999, 3100116777),
 (64, 671860, 'BattleBit Remastered', 146138, 63529, 90.0

### Filter all 5 shared genre tags

In [57]:
#inner_statement = text("SELECT app_id FROM tags WHERE tag IN ('Open World', 'Multiplayer', 'Shooter', 'Difficult', 'Action', 'Gore')")
statement = text(f"""SELECT st.*
FROM steam_twitch_agg AS st
JOIN tags as t
ON st.app_id = t.app_id
WHERE t.tag IN ('Multiplayer', 'FPS', 'Shooter', 'Difficult', 'Action')
GROUP BY st.app_id 
HAVING COUNT(st.app_id) > 4
ORDER BY st.avg_hours_streamed DESC;""")
result = session.execute(statement).all()
games_sharing_5_tags = result
print(len(games_sharing_5_tags))
games_sharing_5_tags

10


[(128, 730, 'Counter-Strike: Global Offensive', 906847, 7382695, 88.0, 48489362, 123006, 66604, 1243, '2012-08-21', 1499, 11066659805),
 (517, 578080, 'PUBG: BATTLEGROUNDS', 495358, 2201296, 57.0, 9725767, 47808, 13337, 679, '2017-12-21', 2999, 6601686704),
 (352, 594650, 'Hunt: Showdown', 117086, 133530, 83.0, 2149006, 10008, 2959, 160, '2019-08-27', 3999, 533986470),
 (146, 782330, 'DOOM Eternal', 105214, 151201, 91.0, 2281359, 15405, 3086, 142, '2020-03-19', 5999, 907054799),
 (391, 232090, 'Killing Floor 2', 42500, 74868, 89.0, 216266, 13175, 297, 58, '2016-11-18', 2999, 224529132),
 (143, 379720, 'DOOM', 35232, 120073, 95.0, 620890, 9235, 842, 47, '2016-05-12', 5999, 720317927),
 (369, 581320, 'Insurgency: Sandstorm', 27906, 85036, 86.0, 612790, 6363, 824, 37, '2018-12-12', 2999, 255022964),
 (277, 493520, 'GTFO', 23939, 34952, 87.0, 851431, 5277, 1155, 32, '2021-12-09', 3999, 139773048),
 (612, 257420, 'Serious Sam 4', 12029, 11809, 83.0, 580536, 2061, 807, 16, '2020-09-24', 3999

## Matching 4+ tags of 5 shared is determined as the same genre

In matching only 3+ tags matched a DRAGON BALL Z fighting game and single player RPG games that are intuitively not the same genre defined by ['Multiplayer', 'FPS', 'Shooter', 'Difficult', 'Action']

In matching all 5 tags, the dataset was filtered too much, and only 10 games resulted

# Save dataframes for visualzation exploration

In [58]:
statement = text("PRAGMA table_info(steam_twitch_agg);")
result = session.execute(statement).all()
columns = [val[1] for val in result]
columns

['index',
 'app_id',
 'title',
 'avg_hours_streamed',
 'reviews_total',
 'review_avg_percent',
 'avg_hours_watched',
 'avg_streamers',
 'avg_viewers',
 'avg_channels',
 'release_date',
 'launch_price_cents',
 'dataset_est_rev_cents']

In [59]:
sharing_4_tags_df = pd.DataFrame(games_sharing_4_tags, columns=columns)
sharing_4_tags_df.head(3)

,index,app_id,title,avg_hours_streamed,reviews_total,review_avg_percent,avg_hours_watched,avg_streamers,avg_viewers,avg_channels,release_date,launch_price_cents,dataset_est_rev_cents
0,127,10,Counter-Strike,913781,137421,97.0,53172684,141632,72571,1248,2000-11-01,999,137283579
1,128,730,Counter-Strike: Global Offensive,906847,7382695,88.0,48489362,123006,66604,1243,2012-08-21,1499,11066659805
2,174,1085660,Destiny 2,565819,555551,82.0,7174193,53869,9832,775,2019-10-01,3499,1943872949


In [60]:
sharing_5_tags_df = pd.DataFrame(games_sharing_5_tags, columns=columns)
sharing_5_tags_df.head(3)

,index,app_id,title,avg_hours_streamed,reviews_total,review_avg_percent,avg_hours_watched,avg_streamers,avg_viewers,avg_channels,release_date,launch_price_cents,dataset_est_rev_cents
0,128,730,Counter-Strike: Global Offensive,906847,7382695,88.0,48489362,123006,66604,1243,2012-08-21,1499,11066659805
1,517,578080,PUBG: BATTLEGROUNDS,495358,2201296,57.0,9725767,47808,13337,679,2017-12-21,2999,6601686704
2,352,594650,Hunt: Showdown,117086,133530,83.0,2149006,10008,2959,160,2019-08-27,3999,533986470


In [61]:
sharing_5_tags_df.to_csv(f'{resources_path}/Hunt-LB_gte5_tags_shared.csv', index=False)
sharing_4_tags_df.to_csv(f'{resources_path}/Hunt-LB_gte4_tags_shared.csv', index=False)

# Close session

In [62]:
session.close()